In [ ]:
import json
import pandas as pd
from datetime import datetime
import os
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
class ModelResultsSaver:
    def __init__(self, model_name='oral_disease_model'):
        self.model_name = model_name
        self.timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.results_dir = f'model_results_{self.timestamp}'
        os.makedirs(self.results_dir, exist_ok=True)
        
    def save_metrics(self, y_true, y_pred, class_names):
        """
        Calculate and save all metrics
        """
        # Calculate metrics
        metrics = {
            'accuracy': float(np.mean(y_true == y_pred)),
            'f1_macro': float(f1_score(y_true, y_pred, average='macro')),
            'f1_weighted': float(f1_score(y_true, y_pred, average='weighted')),
            'precision_macro': float(precision_score(y_true, y_pred, average='macro')),
            'precision_weighted': float(precision_score(y_true, y_pred, average='weighted')),
            'recall_macro': float(recall_score(y_true, y_pred, average='macro')),
            'recall_weighted': float(recall_score(y_true, y_pred, average='weighted'))
        }
        
        # Get per-class metrics
        report_dict = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
        
        # Save metrics as JSON
        with open(f'{self.results_dir}/metrics.json', 'w') as f:
            json.dump({
                'overall_metrics': metrics,
                'per_class_metrics': report_dict
            }, f, indent=4)
        
        # Save metrics as CSV for easy viewing
        df_metrics = pd.DataFrame(report_dict).transpose()
        df_metrics.to_csv(f'{self.results_dir}/metrics.csv')
        
        return metrics, report_dict

    def plot_confusion_matrix(self, y_true, y_pred, class_names):
        """
        Plot and save confusion matrix
        """
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(
            cm, 
            annot=True, 
            fmt='d', 
            cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names
        )
        plt.title('Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig(f'{self.results_dir}/confusion_matrix.png')
        plt.close()
        
        # Save confusion matrix as CSV
        df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
        df_cm.to_csv(f'{self.results_dir}/confusion_matrix.csv')

    def save_training_history(self, history):
        """
        Save training history plots and metrics
        """
        # Save history as JSON
        with open(f'{self.results_dir}/training_history.json', 'w') as f:
            history_dict = {key: [float(val) for val in values] 
                          for key, values in history.history.items()}
            json.dump(history_dict, f, indent=4)
        
        # Plot training history
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Accuracy plot
        ax1.plot(history.history['accuracy'], label='Training')
        ax1.plot(history.history['val_accuracy'], label='Validation')
        ax1.set_title('Model Accuracy')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Accuracy')
        ax1.legend()
        
        # Loss plot
        ax2.plot(history.history['loss'], label='Training')
        ax2.plot(history.history['val_loss'], label='Validation')
        ax2.set_title('Model Loss')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Loss')
        ax2.legend()
        
        plt.tight_layout()
        plt.savefig(f'{self.results_dir}/training_history.png')
        plt.close()

    def generate_report(self, metrics, class_names, model_params=None):
        """
        Generate a comprehensive HTML report
        """
        html_content = f"""
        <html>
        <head>
            <title>Model Evaluation Report</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 20px; }}
                table {{ border-collapse: collapse; width: 100%; }}
                th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
                th {{ background-color: #f2f2f2; }}
                .metric-value {{ font-weight: bold; color: #2c3e50; }}
            </style>
        </head>
        <body>
            <h1>Model Evaluation Report - {self.model_name}</h1>
            <p>Generated on: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}</p>
            
            <h2>Overall Metrics</h2>
            <table>
                <tr><th>Metric</th><th>Value</th></tr>
                <tr><td>Accuracy</td><td class="metric-value">{metrics['accuracy']:.4f}</td></tr>
                <tr><td>F1 Score (Macro)</td><td class="metric-value">{metrics['f1_macro']:.4f}</td></tr>
                <tr><td>F1 Score (Weighted)</td><td class="metric-value">{metrics['f1_weighted']:.4f}</td></tr>
                <tr><td>Precision (Macro)</td><td class="metric-value">{metrics['precision_macro']:.4f}</td></tr>
                <tr><td>Recall (Macro)</td><td class="metric-value">{metrics['recall_macro']:.4f}</td></tr>
            </table>
            
            <h2>Visualizations</h2>
            <p>The following visualizations have been saved:</p>
            <ul>
                <li>Confusion Matrix: confusion_matrix.png</li>
                <li>Training History: training_history.png</li>
            </ul>
            
            <h2>Class Names</h2>
            <ul>
                {' '.join(f'<li>{name}</li>' for name in class_names)}
            </ul>
        </body>
        </html>
        """
        
        with open(f'{self.results_dir}/report.html', 'w') as f:
            f.write(html_content)

# Example usage
def save_all_results(model, history, test_ds, class_names):
    """
    Save all results after model training
    """
    # Initialize results saver
    results_saver = ModelResultsSaver()
    
    # Get predictions
    y_pred = []
    y_true = []
    for images, labels in test_ds:
        pred = model.predict(images)
        pred_classes = np.argmax(pred, axis=1)
        y_pred.extend(pred_classes)
        y_true.extend(labels.numpy())
    
    # Convert to numpy arrays
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    
    # Save all results
    metrics, _ = results_saver.save_metrics(y_true, y_pred, class_names)
    results_saver.plot_confusion_matrix(y_true, y_pred, class_names)
    results_saver.save_training_history(history)
    results_saver.generate_report(metrics, class_names)
    
    print(f"All results saved in directory: {results_saver.results_dir}")

# Use in your main training script:
"""
# After training:
save_all_results(model, history, test_ds, class_names)
"""

In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
import json

In [2]:
# Image parameters
IMG_SIZE = (320, 320)
BATCH_SIZE = 32

In [3]:
# Helper function to preprocess images using OpenCV
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    image = cv2.resize(image, IMG_SIZE)  # Resize to 320x320
    image = image / 255.0  # Normalize to [0, 1]
    return image

In [4]:
# Custom data loader using OpenCV
def load_dataset(directory):
    images = []
    labels = []
    class_names = sorted(os.listdir(directory))  # Ensure consistent ordering
    class_map = {cls: idx for idx, cls in enumerate(class_names)}
    for cls in class_names:
        class_dir = os.path.join(directory, cls)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            images.append(preprocess_image(img_path))
            labels.append(class_map[cls])
    return np.array(images), np.array(labels), class_names

In [5]:
# Load datasets
train_images, train_labels, class_names = load_dataset("dataset/train")
valid_images, valid_labels, _ = load_dataset("dataset/valid")
test_images, test_labels, _ = load_dataset("dataset/test")

# One-hot encode labels
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=len(class_names))
valid_labels = tf.keras.utils.to_categorical(valid_labels, num_classes=len(class_names))
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=len(class_names))

# EfficientNetB0 model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(320, 320, 3))
base_model.trainable = False  # Freeze base model layers

# Custom classification head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    x=train_images, y=train_labels,
    validation_data=(valid_images, valid_labels),
    batch_size=BATCH_SIZE,
    epochs=10
)

# Save the model
model.save("oral_disease_model.keras")

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)
y_true = np.argmax(test_labels, axis=1)
y_pred = np.argmax(predictions, axis=1)

# Metrics
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
conf_matrix = confusion_matrix(y_true, y_pred)

# Save metrics to a file
results = {
    "accuracy": test_acc,
    "classification_report": report,
    "confusion_matrix": conf_matrix.tolist()
}

with open("results.json", "w") as f:
    json.dump(results, f, indent=4)

# Display confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.5094 - loss: 0.7000 - val_accuracy: 0.5111 - val_loss: 0.6994
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.5236 - loss: 0.6942 - val_accuracy: 0.5111 - val_loss: 0.6935
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.5518 - loss: 0.6896 - val_accuracy: 0.5111 - val_loss: 0.6929
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 122s 1s/step - accuracy: 0.5382 - loss: 0.6907 - val_accuracy: 0.5111 - val_loss: 0.6930
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 124s 1s/step - accuracy: 0.5193 - loss: 0.6924 - val_accuracy: 0.5111 - val_loss: 0.6952
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.5441 - loss: 0.6899 - val_accuracy: 0.5111 - val_loss: 0.6936
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.5321 - loss: 0.6918 - val_accuracy: 0.5111 - val_loss: 0.6944
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 124s 1s/step - accuracy: 0.5292 - loss: 0.6919 - val_accu

E:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: Unde

In [6]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
import numpy as np
import json

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
import numpy as np
import json
import os

# Parameters
batch_size = 32
img_size = (320, 320)

# Function to create a dataset from directories
def create_dataset(directory):
    dataset = tf.keras.utils.image_dataset_from_directory(
        directory,
        image_size=img_size,
        batch_size=batch_size
    )
    return dataset

# Load datasets
train_dataset = create_dataset("dataset/train")
valid_dataset = create_dataset("dataset/valid")
test_dataset = create_dataset("dataset/test")

# Determine class names
class_names = train_dataset.class_names

# Prefetch datasets for performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Calculate class weights to handle imbalance
train_labels = np.concatenate([y.numpy() for _, y in train_dataset], axis=0)
class_weights = class_weight.compute_class_weight(
    'balanced', classes=np.unique(train_labels), y=train_labels
)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Class Weights:", class_weights_dict)

# Build the model
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(320, 320, 3))
base_model.trainable = True

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')
])

# Compile the model with learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3, decay_steps=1000, decay_rate=0.9
)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=30,
    class_weight=class_weights_dict,
    callbacks=[early_stopping]
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc}")

# Generate predictions and calculate metrics
y_true = np.concatenate([y.numpy() for _, y in test_dataset], axis=0)
y_pred = np.argmax(model.predict(test_dataset), axis=1)

report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
conf_matrix = confusion_matrix(y_true, y_pred)

# Save results
results = {
    "accuracy": test_acc,
    "classification_report": report,
    "confusion_matrix": conf_matrix.tolist()
}

with open("results.json", "w") as f:
    json.dump(results, f, indent=4)

# Save the trained model
model.save("oral_disease_model.h5")

print("Results and model saved successfully.")

Found 3177 files belonging to 2 classes.
Found 135 files belonging to 2 classes.
Found 145 files belonging to 2 classes.
Class Weights: {0: 0.9289473684210526, 1: 1.0828220858895705}
Epoch 1/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 787s 8s/step - accuracy: 0.9115 - loss: 0.2122 - val_accuracy: 0.8889 - val_loss: 1.1951
Epoch 2/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 729s 7s/step - accuracy: 0.9866 - loss: 0.0451 - val_accuracy: 0.9704 - val_loss: 0.1708
Epoch 3/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 734s 7s/step - accuracy: 0.9835 - loss: 0.0476 - val_accuracy: 0.9778 - val_loss: 0.1691
Epoch 4/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 729s 7s/step - accuracy: 0.9903 - loss: 0.0293 - val_accuracy: 0.8000 - val_loss: 1.6271
Epoch 5/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 725s 7s/step - accuracy: 0.9863 - loss: 0.0393 - val_accuracy: 0.8519 - val_loss: 0.5870
Epoch 6/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 728s 7s/step - accuracy: 0.9855 - loss: 0.0399 - val_accuracy: 0.9630 - val_loss: 0.1412
Epoch 7/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 781